# Greenland peripheral glacier automated terminus detection image download and pre-processing

By Jukes Liu (jukes.liu@boisestate.edu)

_Last modified 05-20-2020._

# 1) Set-up: import necessary packages, set paths, and glaciers of interest

In [3]:
import numpy as np
import pandas as pd
import scipy
import math

import subprocess
import os
import shutil

import cv2
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point
import shapely
import rasterio

# Enable fiona KML file reading driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

#Set base paths
basepath='/home/jukes/Documents/Sample_glaciers/' # folder containing the box shapefile and info
downloadpath = '/media/jukes/jukes1/LS8aws/' # folder to contain the downloaded images

# import necessary functions from automated-glacier-terminus.py
os.chdir('/home/jukes/automated-glacier-terminus') #import necessary functions:
from automated_terminus_functions import distance, resize_pngs

In [60]:
import sys
!{sys.executable} -m pip install --upgrade imutils

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25850 sha256=65cdf17c62ac467474b456c979dc8ca3cee3e14c2c80c0a37e745c9b1551e6c2
  Stored in directory: /home/jukes/.cache/pip/wheels/fc/9c/6d/1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils


#### Enter glaciers of interest by their 3-digit BoxIDs

In [1]:
BoxIDs = ['220', '240', '245', '246', '270', '277', '296']# BoxIDs = np.arange(101, 301, 5)
# # BoxIDs = np.arange(100, 300, 5)
# BoxIDs = list(map(str, BoxIDs))
# BoxIDs.remove('141');
print(BoxIDs)

['220', '240', '245', '246', '270', '277', '296']


#### Create new folders corresponding to these glaciers

In [3]:
# create new BoxID folders 
for BoxID in BoxIDs:
    # create folder in Sample_glaciers
    if os.path.exists(basepath+'Box'+BoxID)==True:
#         shutil.rmtree(basepath+'Box'+BoxID)
        print("Path exists already in Sample_glaciers for Box", BoxID)
    else:
        os.mkdir(basepath+'Box'+BoxID)
        
    # move terminus box shapefile into the new folder
    ID = int(BoxID) # make into an integer in order to grab the .shp files from Boxes_individual
    boxespath = '/media/jukes/jukes1/Boxes_individual/'
    for file in os.listdir(boxespath):
        if file.startswith(str(ID)):
            if len(file) == len(str(ID))+4:
#             if file.endswith('.dbf') or file.endswith('.prj') or file.endswith('.qpj') or file.endswith('.shx') or file.endswith('.shp'):
                shutil.copyfile(boxespath+file, basepath+'Box'+BoxID+'/Box'+BoxID+file[-4:])

    # move RGI glacier outline into the new folder
    RGIpath = '/media/jukes/jukes1/RGI_shps/'
    for file in os.listdir(RGIpath):
        if file.startswith('BoxID_'+str(ID)):
            shutil.copyfile(RGIpath+file, basepath+'Box'+BoxID+'/RGI_Box'+BoxID+file[-4:])
            
    # make new download path folder
    if os.path.exists(downloadpath+'/Box'+BoxID)==True:
        print("Path exists already in LS8aws for Box", BoxID)
    else:
        os.mkdir(downloadpath+'/Box'+BoxID)

Path exists already in Sample_glaciers for Box 220
Path exists already in LS8aws for Box 220
Path exists already in Sample_glaciers for Box 240
Path exists already in LS8aws for Box 240
Path exists already in Sample_glaciers for Box 245
Path exists already in LS8aws for Box 245
Path exists already in Sample_glaciers for Box 246
Path exists already in LS8aws for Box 246
Path exists already in Sample_glaciers for Box 270
Path exists already in LS8aws for Box 270
Path exists already in Sample_glaciers for Box 277
Path exists already in LS8aws for Box 277
Path exists already in Sample_glaciers for Box 296
Path exists already in LS8aws for Box 296


# 2) Find all the Landsat Path Row identifiers that overlap the glaciers

This step requires the WRS-2_bound_world.kml file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/land-resources/nli/landsat/landsat-shapefiles-and-kml-files). To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be reprojected into WRS84 coordinates (ESPG: 4326) using GDAL command line command:

    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER output.shp input.shp

In [9]:
# reproject
for BoxID in BoxIDs:
    boxpath = basepath+"Box"+BoxID+"/Box"+BoxID
            
    rp = "ogr2ogr --config GDAL_DATA /usr/share/gdal/ -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 "+boxpath+"_WRS.shp "+boxpath+".shp"
    subprocess.call(rp, shell=True)
    print(BoxID)

220
240
245
246
270
277
296


In [10]:
# Grab the boxes' WRS84 coordinates
box_points = {}
for BoxID in BoxIDs:
    boxpath = basepath+"Box"+BoxID+"/Box"+BoxID; termbox = fiona.open(boxpath+'_WRS.shp')
    box = termbox.next()
    box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0]  
    points = [] # to hold the vertices
    for coord_pair in box_coords:
        lat = coord_pair[0]; lon = coord_pair[1]
        #create shapely pyoints and append to points list
        point = shapely.geometry.Point(lat, lon)
#         print(point)
        points.append(point)
    box_points.update({BoxID: points}); print("Box"+BoxID)

Box220
Box240
Box245
Box246
Box270
Box277
Box296


/home/jukes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  """


In [11]:
#open the kml file with the pathrow bounds as WRS
WRS = fiona.open(basepath+'WRS-2_bound_world.kml', driver='KML')

### Loop through all the scene footprints and stores those that overlap ALL of your shapefile vertices:

In [12]:
#create lists to hold the paths and rows and BoxIDs
paths = []; rows = []; boxes = []

#loop through all features in the WRS .kml:
for feature in WRS:
    #create shapely polygons with the pathrow bounds
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
#     print(coords)
    pathrow_poly = Polygon(coords)
    #grab the path and row name
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
    
    #for each feature, loop through each of the box_points
    for BoxID in box_points:
        #create a counter for number of box_points in the pathrow_geom:
        box_points_in = 0
        
        points = box_points.get(BoxID)
        for i in range(0, len(points)):
            point = points[i]
            #if the pathrow shape contains the point
            if point.within(pathrow_poly):
#                 print("Found")
                #append the counter
                box_points_in = box_points_in+1
#         print(box_points_in)
        if box_points_in == 5:
            #save that path row and boxID
            paths.append('%03d' % int(path)); rows.append('%03d' % int(row))
            boxes.append(BoxID)

boxes_pr_df = pd.DataFrame(list(zip(boxes, paths, rows)), columns=['BoxID','Path', 'Row'])
boxes_pr_df = boxes_pr_df.sort_values(by='BoxID')
boxes_pr_df

,BoxID,Path,Row
0,220,232,016
4,220,233,015
10,220,232,015
1,240,231,016
5,240,233,015
11,240,232,015
12,245,232,015
6,245,233,015
16,245,231,015
2,245,231,016


#### Write it to a csv file:

In [17]:
PR_FILENAME = 'LS_pathrows_SE_focus.csv'
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',')

# 3) Download metadata (MTL.txt) files for all available images over glaciers

Bulk download Landsat-8 images and metadata stored on the Amazon Web Services cloud (s3 bucket) over a region of interest using the Amazon Web Services command line interface. Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get aws commands onto your command terminal. The syntax for grabbing an individual metadata file is as follows:

     aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/path/row/LC08_XXXX_pathrow_yyyyMMdd_01_T1/LC08_XXXX_pathrow_yyyyMMdd_01_T1_MTL.txt /path_to/output/

Access https://docs.opendata.aws/landsat-pds/readme.html to learn more.

#### Download the metadatafiles into Path_Row folders created using:

    aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt" 

In [19]:
boxes_pr_df = pd.read_csv(basepath+PR_FILENAME, dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); boxes_pr_df

,Unnamed: 0,Path,Row
BoxID,,,
220,0,232,016
220,4,233,015
220,10,232,015
240,1,231,016
240,5,233,015
240,11,232,015
245,12,232,015
245,6,233,015
245,16,231,015


In [20]:
#Loop through the dataframe with your path row combinations:
for index, row in boxes_pr_df.iterrows():
    #grab the path row names and set the folder name
    path = row['Path']; row = row['Row']; folder_name = 'Path'+path+'_Row'+row+'_c1'

    #set path to access the image on the amazon cloud:
    bp_in = 's3://landsat-pds/c1/L8/'; totalp_in = bp_in+path+'/'+row+'/'
    #set output path for the downloaded files:
    bp_out = downloadpath+folder_name+'/'; print(bp_out)
    
    #create Path_Row folders and do the download if they don't exist already to hold the metadata
    if os.path.exists(bp_out):
        print(folder_name, " EXISTS ALREADY. SKIP.")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")

        command = 'aws --no-sign-request s3 cp '+totalp_in+' '+bp_out+' --recursive --exclude "*" --include "*MTL.txt"'
        print(command)

        #call the command line that downloads the metadata files using aws
        subprocess.call(command, shell=True)

/media/jukes/jukes1/LS8aws/Path232_Row016_c1/
Path232_Row016_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path233_Row015_c1/
Path233_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path232_Row015_c1/
Path232_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path231_Row016_c1/
Path231_Row016_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path233_Row015_c1/
Path233_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path232_Row015_c1/
Path232_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path232_Row015_c1/
Path232_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path233_Row015_c1/
Path233_Row015_c1  EXISTS ALREADY. SKIP.
/media/jukes/jukes1/LS8aws/Path231_Row015_c1/
Path231_Row015_c1 directory made
aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/231/015/ /media/jukes/jukes1/LS8aws/Path231_Row015_c1/ --recursive --exclude "*" --include "*MTL.txt"
/media/jukes/jukes1/LS8aws/Path231_Row016_c1/
Path231_Row016_c1  EXISTS 

# 4) Determine how cloudy the images are using the LS quality band files subset to terminus box

Use gdalwarp commands and the __vsicurl__ link to download subset of the quality band we will use to determine cloud cover:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/031/005/imagename/imagename_BQA.TIF path_to_subsetBQA.TIF
    
These will need to go into the newly created BoxID folders because the subset of the same image for different boxes will be different. We will need to reproject the shapefiles into UTM to match the LS8 bands. We must grab the UTM zones from the metadata files downloaded and fill in the following syntax:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone -s_srs EPSG:3413 output.shp input.shp

In [21]:
zones = {}; zone_df = []

#Loop through the dataframe with your path row combinations:
for index, row in boxes_pr_df.iterrows():
    path = row['Path']; row = row['Row']; BoxID = str(index); folder_name = 'Path'+path+'_Row'+row+'_c1'
    
    #set path to path row folders to grab scenes from and output path according to each of the BoxIDs
    pr_folderpath = downloadpath+folder_name+'/'
    
    #set shapefile path for each BoxID
    pathtoshp = basepath+"Box"+BoxID+"/Box"+BoxID
    
    #if there are files in the folder:
    if len(os.listdir(pr_folderpath)) > 0:
        #grab UTM Zone from the first metadata file in the pr_folderpath
        mtl_scene = os.listdir(pr_folderpath)[0]
        mtl = open(pr_folderpath+mtl_scene+'/'+mtl_scene+'_MTL.txt', 'r')
        
        #loop through each line in metadata to find the UTM ZONE
        for line in mtl:
            variable = line.split("=")[0]
            if ("UTM_ZONE" in variable): 
                #save it:
                zone = '%02d' % int(line.split("=")[1][1:-1])
                zones.update({folder_name: zone}); zone_df.append(zone)
                
        #reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326'+zone+' -s_srs EPSG:4326 '+pathtoshp+"_UTM_"+zone+".shp "+pathtoshp+"_WRS.shp"
        subprocess.call(rp_shp, shell=True)
    #if not, append nan to zone and do nothing
    else:
        zone_df.append(np.nan)
        
    #grab the names of the LS8 scenes from the downloaded metadatafiles using os.listdir() in the Path_Row folders
    scenes = os.listdir(pr_folderpath)

    for scene in scenes:
        #if the file grabbed does correspond to a Landsat scene (starts with LC08):
        if len(scenes) > 0 and scene.startswith("LC08"):            
            #set path to reprojected box shp
            pathtoshp_rp = pathtoshp+'_UTM_'+zones[folder_name]
            
            #set path to the quality band (BQA.TIF) in the cloud using the scenename
            pathtoBQA = '/vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/'+path+'/'+row+'/'+scene+"/"+scene+"_BQA.TIF"
            subsetout = pr_folderpath+scene+"/"+scene+'_BQA_Box'+BoxID+'.TIF'

            #Check download command syntax
            BQA_dwnld_cmd = 'gdalwarp -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '+pathtoBQA+' '+subsetout

            #When you've checked everything, uncomment the following to run:
            subprocess.call(BQA_dwnld_cmd, shell=True)
            print(scene+'_BQA_Box'+BoxID+'.TIF subset downloaded')

boxes_pr_df['Zone'] = zone_df
print("Done.")

LC08_L1TP_232016_20181129_20181129_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20181012_20181012_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20190929_20191017_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20190508_20190508_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20130912_20170502_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20190116_20190116_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20200424_20200424_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20170518_20170518_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20180419_20180419_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20161107_20170318_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20181113_20181113_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20190321_20190321_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20190727_20190727_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20170705_20170705_01_

LC08_L1TP_232016_20150801_20170406_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20200408_20200422_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20170907_20170907_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20130928_20180528_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_232016_20180521_20180521_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20141001_20170418_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_232016_20190625_20190625_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20181120_20181120_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20150605_20170408_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20180410_20180410_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20160607_20170324_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20190803_20190819_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20180901_20180901_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20150520_20170408_01_

LC08_L1TP_233015_20181104_20181104_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20190224_20190224_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20190819_20190819_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20180917_20180917_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20160725_20170322_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_233015_20190718_20190718_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20171016_20171018_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20191107_20191107_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20170806_20170813_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20191015_20191015_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20170721_20170721_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1GT_232015_20180521_20180521_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20130928_20180528_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_232015_20180825_20180825_01_

LC08_L1GT_232015_20180708_20180708_01_RT_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20190625_20190705_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1TP_232015_20150630_20170407_01_T1_BQA_Box220.TIF subset downloaded
LC08_L1GT_233015_20181120_20181120_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20150605_20170408_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1GT_233015_20180410_20180410_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20160607_20170324_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20190803_20190819_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20180901_20180901_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20150520_20170408_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20170829_20170829_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20150723_20170406_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20170829_20170914_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20171117_20171117_01_

LC08_L1TP_233015_20160725_20170322_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_233015_20190718_20190718_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1GT_233015_20171016_20171018_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1GT_233015_20191107_20191107_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20170806_20170813_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20191015_20191015_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20170721_20170721_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1GT_232015_20180521_20180521_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20130928_20180528_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1GT_232015_20180825_20180825_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20190305_20190305_01_RT_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20180926_20181009_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20160718_20170323_01_T1_BQA_Box240.TIF subset downloaded
LC08_L1TP_232015_20171110_20171121_01_

...

LC08_L1TP_001014_20140812_20170420_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20141031_20170418_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20190215_20190215_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20171007_20171007_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20171023_20171107_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20170804_20170812_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20150628_20170407_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20181111_20181111_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20180924_20180924_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20190709_20190709_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20200422_20200422_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1GT_001014_20180401_20180401_01_RT_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20190709_20190719_01_T1_BQA_Box277.TIF subset downloaded
LC08_L1TP_001014_20170601_20170602_01_

LC08_L1TP_231014_20130414_20170505_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20141010_20170418_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20170831_20170831_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1GT_231014_20180412_20180412_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20160406_20170327_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1GT_231014_20180701_20180701_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20191024_20191030_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20190210_20190210_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20150623_20170407_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20200229_20200229_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231014_20181021_20181031_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1GT_231014_20190314_20190314_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1GT_231014_20171018_20171019_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1GT_231014_20190517_20190517_01_

LC08_L1GT_230014_20190611_20190611_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20180608_20180608_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20140731_20170420_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20161008_20170320_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20170520_20170525_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20200426_20200426_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1GT_230014_20170824_20170824_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20190510_20190521_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20180507_20180517_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20160922_20170321_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20190323_20190323_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20200512_20200512_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20170909_20170927_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_230014_20180608_20180615_01_

LC08_L1TP_232013_20170603_20170615_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20170619_20170619_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20160413_20170326_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20141102_20180528_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20170227_20170316_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20181012_20181012_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20170619_20170629_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20170518_20170518_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20160515_20170324_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20150801_20170406_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1GT_232013_20181113_20181113_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20180622_20180703_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20190727_20190727_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_232013_20191015_20191029_01_

LC08_L1TP_231013_20180818_20180829_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20181021_20181021_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20140620_20170421_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1GT_231013_20190226_20190226_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20170308_20170317_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20181106_20181106_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20170628_20170628_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1GT_231013_20171018_20171019_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20191024_20191024_01_RT_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20150623_20170407_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20191024_20191030_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20170511_20170525_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20140519_20170422_01_T1_BQA_Box296.TIF subset downloaded
LC08_L1TP_231013_20170714_20170714_01_

#### Re-save csv file with the zone information

In [22]:
PR_FILENAME = 'LS_pathrows_SE_focus.csv'
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',')

# 5) Create buffer zone around terminus boxes and rasterize/subset terminus boxes (used for cropping and masknig later on)

The following code pulls the buffer distances around the terminus boxes from an existing .csv file with the exported attributes tables for the peripheral glacier terminus boxes. These buffer distances will be used to create a buffer zone to subset the Landsat scenes.

In [23]:
buffers = []
#Calculate a buffer distance around the terminus box using the UTM projected boxes
for BoxID in BoxIDs:
    buff_distances = []

    for file in os.listdir(basepath+'Box'+BoxID+'/'):
        if 'UTM' in file and '.shp' in file and "Box" in file:
            print(file)
            boxpath = basepath+"Box"+BoxID+"/"+file  
            termbox = fiona.open(boxpath)
            #grab the box feature:
            box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0]
            points = []
            for coord_pair in box_coords:
                lat = coord_pair[0]; lon = coord_pair[1]; points.append([lat, lon])
            
            #Calculate distance between 1 and 2 and distance between 2 and 3 and pick the longer one
            coord1 = points[0]; coord2 = points[1]; coord3 = points[2]   
            dist1 = distance(coord1[0], coord1[1], coord2[0], coord2[1]);
            dist2 = distance(coord2[0], coord2[1], coord3[0], coord3[1])  
            buff_dist = int(np.max([dist1, dist2]))
            buff_distances.append(buff_dist) 
    buffer = buff_distances[0]; buffers.append(buffer)
buff_df = pd.DataFrame(list(zip(BoxIDs, buffers)), columns=['BoxID', 'Buff_dist_m'])
buff_df

Box220_UTM_24.shp
Box240_UTM_24.shp
Box245_UTM_24.shp
Box246_UTM_24.shp
Box270_UTM_24.shp
Box277_UTM_24.shp
Box296_UTM_25.shp
Box296_UTM_24.shp


/home/jukes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  if sys.path[0] == '':


,BoxID,Buff_dist_m
0,220,1824
1,240,2665
2,245,1695
3,246,1356
4,270,1432
5,277,1379
6,296,1291


The next section creates a buffer zone shapefile using GDAL command **ogr2ogr** with the following syntax:

    ogr2ogr Buffer###.shp path_to_terminusbox###.shp  -dialect sqlite -sql "SELECT ST_Buffer(geometry, buffer_distance) AS geometry,*FROM 'Box###'" -f "ESRI Shapefile"

In [24]:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    buff_dist = str(row['Buff_dist_m'])
    
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    outputbuffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #SET buffer command and print to check it
    buffer_cmd = 'ogr2ogr '+outputbuffer_path+" "+terminusbox_path+' -dialect sqlite -sql "SELECT ST_Buffer(geometry, '+buff_dist+") AS geometry,*FROM 'Box"+BoxID+"'"+'" -f "ESRI Shapefile"'
    print(buffer_cmd)
    
    subprocess.call(buffer_cmd, shell=True) 
    print("Box"+BoxID)

ogr2ogr /home/jukes/Documents/Sample_glaciers/Box220/Buffer220.shp /home/jukes/Documents/Sample_glaciers/Box220/Box220.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 1824) AS geometry,*FROM 'Box220'" -f "ESRI Shapefile"
Box220
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box240/Buffer240.shp /home/jukes/Documents/Sample_glaciers/Box240/Box240.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 2665) AS geometry,*FROM 'Box240'" -f "ESRI Shapefile"
Box240
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box245/Buffer245.shp /home/jukes/Documents/Sample_glaciers/Box245/Box245.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 1695) AS geometry,*FROM 'Box245'" -f "ESRI Shapefile"
Box245
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box246/Buffer246.shp /home/jukes/Documents/Sample_glaciers/Box246/Box246.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 1356) AS geometry,*FROM 'Box246'" -f "ESRI Shapefile"
Box246
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box270/Buffer270.shp /

The terminus box shapefiles are then rasterized (to be used as a mask during the WTMM filering) using the GDAL **gdal_rasterize** command and subset to the buffer zone using the GDAL **gdalwarp** command using the following syntax:

1) Rasterize

    gdal_rasterize -burn 1.0 -tr x_resolution y_resolution -a_nodata 0.0 path_to_terminusbox.shp path_to_terminusbox_raster.TIF

The x_resolution and y_resolution are set to be 15.0 (meters) to match the Landsat B8 resolution.
    
2) Subset

    gdalwarp -cutline path_to_Buffer###.shp -crop_to_cutline path_to_terminusbox_raster.TIF path_to_subset_raster_cut.TIF

In [25]:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #output raster path:
    terminusraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+".TIF"
    cutraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+"_raster_cut.TIF"
    
    #SET commands and print to check
    rasterize_cmd = 'gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 '+terminusbox_path+' '+terminusraster_path
    subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+' -crop_to_cutline '+terminusraster_path+" "+cutraster_path
    
    #RASTERIZE & SUBSET
    subprocess.call(rasterize_cmd, shell=True)
    subprocess.call(subsetbuffer_cmd, shell=True)
    
    print("Box"+BoxID)

Box220
Box240
Box245
Box246
Box270
Box277
Box296


# 6) Download non-cloudy images

To threshold out cloudy images, we will find the number of pixels in our shapefile region that correspond to a cumulative pixel value of > __25000__. If the number of pixels with values above this threshold is greater than our % threshold, we won't download the image. 

__EDITED: We'll also filter out those images along the border that cutoff through your shapefile region. These will have a designated fill value of 1.__

Set cloudy and fill thresholds:

In [26]:
#SET cloud cover threshold
cpercent_thresh = 20.0

#SET fill percent threshold
fpercent_thresh = 60.0

#SET BQA pixel value threshold
BQA_thresh = 4096.0
# use 4096?

Set LS bands to download:

In [27]:
#SET your desired bands here
bands = [8]
# bands = [2, 3, 4]

### Download the images that pass these thresholds:

In [28]:
#Loop through the dataframe with your path row combinations:
for index, row in boxes_pr_df.iterrows():
    #grab the path row names and set the folder name
    path = row['Path']; zone = row['Zone']; row = row['Row']; BoxID = index; 
    folder_name = 'Path'+path+'_Row'+row+'_c1'
    
    #set path to path row folders to grab BQA files from
    pr_folderpath = downloadpath+folder_name+'/'
    
    #Output BoxID folderpath and create BoxID folders
    bp_out = downloadpath+'Box'+BoxID+'/'
    
    if os.path.exists(bp_out):
        print("Box"+BoxID, " EXISTS ALREADY. SKIP.")
    else:
        os.mkdir(bp_out)
        print("Box"+BoxID+" directory made")
                      
    #set path to the buffer zones for final subsetting:
    pathtobuffer = basepath+'Box'+BoxID+'/Buffer'+BoxID
#     pathtoshp_rp = basepath+'Box'+BoxID+'/Box'+BoxID+'_UTM_'+zone+'.shp'
    
    #REPROJECT BUFFER SHAPE
    if len(os.listdir(pr_folderpath)) > 0:
#         print(zone)
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326'+zone+' -s_srs EPSG:3413 '+pathtobuffer+"_UTM_"+zone+".shp "+pathtobuffer+".shp"
#         print(rp_shp)
        subprocess.call(rp_shp, shell=True)

    #loop through all the BQA files in the path_row folders:
    for scene in os.listdir(pr_folderpath):
        if len(os.listdir(pr_folderpath)) > 0 and scene.startswith("LC08"):
            #grab all the BQA files to calculate the percent cloud
            BQApath = pr_folderpath+scene+"/"+scene+'_BQA_Box'+BoxID+'.TIF'
#             print(BQApath)
            
            #read in your BQA subset raster as a numpy array:
            subsetBQA = mpimg.imread(BQApath)
            
            #CALCULATE % CLOUD COVER USING YOUR BQA SUBSET
            #first, calculate the total number of pixels
            totalpixels = subsetBQA.shape[0]*subsetBQA.shape[1]
            
            #then, calculate the number of cloud pixels:
            #mask for pixels that have a value > than your threshold value
            cloudBQA = subsetBQA[subsetBQA > BQA_thresh]
            
            #calculate number of fill pixels (fill value of 1. Will be less than 2.0)
            fillBQA = subsetBQA[subsetBQA < 2.0]
            
            #and count those cloudpixels and fillpixels:
            cloudpixels = len(cloudBQA); fillpixels = len(fillBQA)
            
            #divide this by the totalpixels and multiply by 100
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100)
            fillpercent = int(float(fillpixels)/float(totalpixels)*100)
            
            #check that the values are sensible
            print(scene, 'Cloud % ', cloudpercent, 'Fill %', fillpercent)
            
            #set path to rp buffer:
            pathtobuffer_rp = pathtobuffer+"_UTM_"+zone+".shp"
            
            #IF % CLOUD COVER IS LESS THAN OR EQUAL TO YOUR THRESHOLD DOWNLOAD THE BANDS FOR THAT SCENE:
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                #download the bands for that scene into your scene folders:
                for band in bands:
                    #make sure the band is in string format:
                    band = str(band)
                    #set input path to your bands in AWS:
                    pathin = '/vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/'+path+"/"+row+"/"+scene+"/"+scene+"_B"+band+".TIF"
                    #set your output path to the overall BoxID folder, not the scene folders
                    pathout = downloadpath+'Box'+BoxID+'/'+scene+"_B"+band+'_Buffer'+BoxID+'.TIF'
                    
                    #set download command and check:
                    download_cmd = 'gdalwarp -cutline '+pathtobuffer_rp+' -crop_to_cutline '+pathin+' '+pathout
#                     download_cmd = 'gdalwarp -cutline '+pathtoshp_rp+' -crop_to_cutline '+pathin+' '+pathout
                    print(download_cmd)
                    
                    #When you're ready, comment out the above print command and
                    #uncomment the following to commence the download:                   
                    subprocess.call(download_cmd, shell=True)
                    print(scene+"_B"+band+'_Buffer'+BoxID+".TIF downloaded")

Box220  EXISTS ALREADY. SKIP.
LC08_L1TP_232016_20181129_20181129_01_RT Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20181129_20181129_01_RT/LC08_L1TP_232016_20181129_20181129_01_RT_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20181129_20181129_01_RT_B8_Buffer220.TIF
LC08_L1TP_232016_20181129_20181129_01_RT_B8_Buffer220.TIF downloaded
LC08_L1GT_232016_20181012_20181012_01_RT Cloud %  1 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1GT_232016_20181012_20181012_01_RT/LC08_L1GT_232016_20181012_20181012_01_RT_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1GT_232016_20181012_20181012_01_RT_B8_Buffer220.TIF
LC08_L1GT_232016_20181012_20181012_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_2019092

LC08_L1GT_232016_20190524_20190604_01_T2_B8_Buffer220.TIF downloaded
LC08_L1GT_232016_20190422_20190422_01_RT Cloud %  56 Fill % 43
LC08_L1TP_232016_20200307_20200314_01_T1 Cloud %  56 Fill % 43
LC08_L1TP_232016_20180302_20180302_01_T1 Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20180302_20180302_01_T1/LC08_L1TP_232016_20180302_20180302_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20180302_20180302_01_T1_B8_Buffer220.TIF
LC08_L1TP_232016_20180302_20180302_01_T1_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20200220_20200220_01_RT Cloud %  56 Fill % 43
LC08_L1TP_232016_20190711_20190719_01_T1 Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20190711_20190719_01_T1/LC08_

LC08_L1GT_232016_20180318_20180318_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20171110_20171110_01_RT Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20171110_20171110_01_RT/LC08_L1TP_232016_20171110_20171110_01_RT_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20171110_20171110_01_RT_B8_Buffer220.TIF
LC08_L1TP_232016_20171110_20171110_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20140729_20170420_01_T1 Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20140729_20170420_01_T1/LC08_L1TP_232016_20140729_20170420_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20140729_20170420_01_T1_B8_Buffer220.TIF
LC08_L1TP_232016_20140729_20170420_01_T1_B8_Buffer220.

LC08_L1TP_232016_20151004_20170403_01_T1_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20190812_20190812_01_RT Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20190812_20190812_01_RT/LC08_L1TP_232016_20190812_20190812_01_RT_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20190812_20190812_01_RT_B8_Buffer220.TIF
LC08_L1TP_232016_20190812_20190812_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20171009_20171024_01_T1 Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20171009_20171024_01_T1/LC08_L1TP_232016_20171009_20171024_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20171009_20171024_01_T1_B8_Buffer220.TIF
LC08_L1TP_232016_20171009_20171024_01_T1_B8_Buffer220.

LC08_L1TP_232016_20191015_20191015_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20170806_20170806_01_RT Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20170806_20170806_01_RT/LC08_L1TP_232016_20170806_20170806_01_RT_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20170806_20170806_01_RT_B8_Buffer220.TIF
LC08_L1TP_232016_20170806_20170806_01_RT_B8_Buffer220.TIF downloaded
LC08_L1TP_232016_20180622_20180623_01_RT Cloud %  56 Fill % 43
LC08_L1TP_232016_20170603_20170615_01_T1 Cloud %  0 Fill % 43
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box220/Buffer220_UTM_24.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/232/016/LC08_L1TP_232016_20170603_20170615_01_T1/LC08_L1TP_232016_20170603_20170615_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box220/LC08_L1TP_232016_20170603_20170615_01_T1_B8_Buffe

...

LC08_L1TP_231013_20140620_20170421_01_T1_B8_Buffer296.TIF downloaded
LC08_L1GT_231013_20190226_20190226_01_RT Cloud %  95 Fill % 4
LC08_L1TP_231013_20170308_20170317_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20170308_20170317_01_T1/LC08_L1TP_231013_20170308_20170317_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20170308_20170317_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20170308_20170317_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20181106_20181106_01_RT Cloud %  95 Fill % 4
LC08_L1TP_231013_20170628_20170628_01_RT Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20170628_20170628_01_RT/LC08_L1TP_231013_20170628_20170628_01_RT_B8.TIF /media/jukes/jukes1/LS8a

LC08_L1TP_231013_20160929_20170321_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20180919_20180928_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20180919_20180928_01_T1/LC08_L1TP_231013_20180919_20180928_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20180919_20180928_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20180919_20180928_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20140604_20170422_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20140604_20170422_01_T1/LC08_L1TP_231013_20140604_20170422_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20140604_20170422_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20140604_20170422_01_T1_B8_Buffer296.TI

LC08_L1TP_231013_20170628_20180528_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20190618_20190703_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20190618_20190703_01_T1/LC08_L1TP_231013_20190618_20190703_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20190618_20190703_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20190618_20190703_01_T1_B8_Buffer296.TIF downloaded
LC08_L1GT_231013_20190314_20190314_01_RT Cloud %  95 Fill % 4
LC08_L1TP_231013_20130905_20170502_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20130905_20170502_01_T1/LC08_L1TP_231013_20130905_20170502_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20130905_20170502_01_T1_B8_Buffer29

LC08_L1TP_231013_20140722_20170421_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20150709_20170407_01_T1 Cloud %  0 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20150709_20170407_01_T1/LC08_L1TP_231013_20150709_20170407_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20150709_20170407_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20150709_20170407_01_T1_B8_Buffer296.TIF downloaded
LC08_L1TP_231013_20160508_20170325_01_T1 Cloud %  9 Fill % 4
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box296/Buffer296_UTM_25.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/231/013/LC08_L1TP_231013_20160508_20170325_01_T1/LC08_L1TP_231013_20160508_20170325_01_T1_B8.TIF /media/jukes/jukes1/LS8aws/Box296/LC08_L1TP_231013_20160508_20170325_01_T1_B8_Buffer296.TIF
LC08_L1TP_231013_20160508_20170325_01_T1_B8_Buffer296.TI

LC08_L1TP_231013_20190720_20190731_01_T1_B8_Buffer296.TIF downloaded


### Reproject newly downloaded images into Greenland Polar Stereo projection

In [29]:
#Loop through the BoxIDs in the dataframe with your path row combinations:
for BoxID in list(set(boxes_pr_df.index)):
   
    #Output BoxID path 
    bp_out = downloadpath+'Box'+BoxID+'/'
    
    #create output folder to reproject into
    if os.path.exists(bp_out+'reprojected/'):
        print("Box"+BoxID, "Reprojected EXISTS ALREADY.")
    else:
        os.mkdir(bp_out+'reprojected/')
        print("Box"+BoxID+" Reprojected directory made")
                      
    #grab all downloaded images and reproject:
    downloadedimages = os.listdir(bp_out)
    for image in downloadedimages:
        if image.startswith("LC08"):
            imagename = image[:-4]
#             os.remove(bp_out+image) # remove old images
    #         print(imagename)
            rp_cmd = "gdalwarp -t_srs '+proj=stere +lat_ts=7|0 +lat_0=90 +lon_0=-45 +y=0 +x=0 +k=1 +datum=WGS84 +units=m' "+bp_out+image+" "+bp_out+'reprojected/'+imagename+"_PS.TIF"
#             print(rp_cmd)
            subprocess.call(rp_cmd, shell=True)   
    print("Box"+BoxID+" reprojected.")

Box245 Reprojected EXISTS ALREADY.
Box245 reprojected.
Box296 Reprojected EXISTS ALREADY.
Box296 reprojected.
Box220 Reprojected EXISTS ALREADY.
Box220 reprojected.
Box240 Reprojected EXISTS ALREADY.
Box240 reprojected.
Box277 Reprojected EXISTS ALREADY.
Box277 reprojected.
Box270 Reprojected EXISTS ALREADY.
Box270 reprojected.
Box246 Reprojected EXISTS ALREADY.
Box246 reprojected.


### Grab the image acquisition dates from the metadata files

In [30]:
import datetime

#create a list of datetime objects for the images:
datetimes = []; scenes_dated = []

#Loop through the dataframe with your path row combinations:
# for index, row in pr_df_multi.iterrows():
for index, row in boxes_pr_df.iterrows():
    path = row['Path']; row = row['Row']; BoxID = index; 
    folder_name = 'Path'+path+'_Row'+row+'_c1'; print(folder_name)
    
    #Output folder paths
    folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath+'Box'+BoxID+'/reprojected/'
    
    #keep track of the number of scenes:
    scenecount = 0
    
    #list of files in directory where you downloaded images
    downloaded_scenes = os.listdir(bp_out)
    for scene in downloaded_scenes:
        #if it's a downloaded image and in the folder we are working on:
        if scene.endswith('.TIF') and scene.startswith("LC08"):
            scenename = scene[:-20]
            if scenename in os.listdir(folderpath):
                #go into the folder and open the metadata file:
                scenefiles = os.listdir(folderpath+scenename+'/')
                for file in scenefiles:
                    if ("MTL.txt" in file):
                        mdata = open(folderpath+scenename+"/"+scenename+"_MTL.txt", "r")
                        #loop through each line in metadata to find the date (and time) of acquisition
                        for line in mdata:
                            variable = line.split("=")[0]
                            if ("DATE_ACQUIRED" in variable):
                                date = line.split("=")[1][1:-1]

                        #save the date as a datetime object and append to list
                        dates = datetime.datetime.strptime(date, '%Y-%m-%d')
                        datetimes.append(dates); scenes_dated.append(scenename)
                scenecount = scenecount+1

# Store in a dataframe
datetime_df = pd.DataFrame(list(zip(scenes_dated, datetimes)), columns=['Scene', 'datetime'])
datetime_df = datetime_df.sort_values(by='datetime', ascending=True); datetime_df = datetime_df.drop_duplicates()
datetime_df

Path232_Row016_c1
Path233_Row015_c1
Path232_Row015_c1
Path231_Row016_c1
Path233_Row015_c1
Path232_Row015_c1
Path232_Row015_c1
Path233_Row015_c1
Path231_Row015_c1
Path231_Row016_c1
Path232_Row015_c1
Path231_Row016_c1
Path233_Row015_c1
Path233_Row015_c1
Path232_Row015_c1
Path233_Row015_c1
Path232_Row015_c1
Path001_Row014_c1
Path231_Row014_c1
Path230_Row014_c1
Path232_Row013_c1
Path231_Row013_c1


,Scene,datetime
1074,LC08_L1TP_231014_20130320_20170505_01_T1,2013-03-20
18,LC08_L1TP_232016_20130402_20170505_01_T1,2013-04-02
1296,LC08_L1TP_231013_20130414_20170505_01_T1,2013-04-14
1070,LC08_L1TP_231014_20130414_20170505_01_T1,2013-04-14
1083,LC08_L1TP_231014_20130430_20170505_01_T1,2013-04-30
...,...,...
141,LC08_L1TP_233015_20200501_20200501_01_RT,2020-05-01
1246,LC08_L1TP_232013_20200510_20200510_01_RT,2020-05-10
345,LC08_L1TP_232015_20200510_20200510_01_RT,2020-05-10
62,LC08_L1TP_232016_20200510_20200510_01_RT,2020-05-10


#### Write image dates to a csv file:

In [31]:
DATES_NAME = 'imgdates_SE_focus.csv'
datetime_df.to_csv(path_or_buf = basepath+DATES_NAME, sep=',')

# 7) Remove cloudy images that slipped through BQA filtering

In [32]:
#convert all files in reprojected folder to png from TIF
for BoxID in BoxIDs:
    print(BoxID)
    command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *_PS.TIF'
#     print(command)
    subprocess.call(command, shell=True)

220
240
245
246
270
277
296


In [33]:
import cv2

for BoxID in BoxIDs:
    imagepath = downloadpath+'Box'+BoxID+'/reprojected/'
    for img in os.listdir(imagepath):
        if img.endswith('Buffer'+BoxID+'_PS.png'):
            image = cv2.imread(imagepath+img,-1)
            imageplt = mpimg.imread(imagepath+img)
            image_nofill = imageplt[imageplt > 0] # don't consider the fill points
            img_std = np.std(image_nofill)
            img_range = np.max(image_nofill) - np.min(image_nofill)
            img_med = np.median(image_nofill)
#             print(img, img_std, img_range, img_med)
            
            if img_std < 0.04 and img_med > 0.15:
                os.remove(imagepath+img) # remove png mimage
                os.remove(imagepath+img[:-4]+'.TIF') # remove pgm image
#                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#                 plt.show()

# 7) Grab fraction of total images available that were cloudy

In [35]:
boxes_pr_df = pd.read_csv(basepath+'LS_pathrows_SE_focus.csv', dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); BoxIDs = list(set(boxes_pr_df.index))
print(BoxIDs); boxes_pr_df.head()

['245', '296', '220', '240', '277', '270', '246']


,Unnamed: 0,Path,Row,Zone
BoxID,,,,
220,0,232,016,24
220,4,233,015,24
220,10,232,015,24
240,1,231,016,NaN
240,5,233,015,24


In [36]:
im_tots = []; downloaded = []; fractions = [];

for BoxID in BoxIDs:
    pathrows_BoxID = boxes_pr_df[boxes_pr_df.index == BoxID].copy() # grab path rows for that BoxID
    
    im_tot = 0 # count number of total images
    for idx, rw in pathrows_BoxID.iterrows():
        p = rw['Path']; r = rw['Row']
        ims_pr = len(os.listdir(downloadpath+'Path'+p+'_Row'+r+'_c1')) # grab number of scenes in that pathrow
        im_tot = im_tot + ims_pr
    
    counter = 0
    if im_tot == 0:
        download_frac = np.NaN
    else:
        # find number downloaded (not cloudy) from the BoxID folder
        for file in os.listdir(downloadpath+'Box'+BoxID+'/reprojected/'):
            if file.startswith('LC08') and file.endswith('.png') and 'B8' in file:
                counter = counter + 1
            
        download_frac = int(counter/im_tot*100) # grab fraction downloaded
        
    im_tots.append(im_tot); downloaded.append(counter); fractions.append(download_frac)
    
downloaded_df = pd.DataFrame(list(zip(BoxIDs, im_tots, downloaded, fractions)), columns = ['BoxID', 'Total_ims', 'Downloaded', '%'])
downloaded_df 

,BoxID,Total_ims,Downloaded,%
0,245,218,138,63
1,296,473,248,52
2,220,336,209,62
3,240,218,134,61
4,277,312,205,65
5,270,218,132,60
6,246,218,136,62


#### Write information on number of cloudy images to a csv file

In [37]:
CLOUDY_NAME = 'Images_downloaded_SE_focus.csv'
downloaded_df.to_csv(basepath+CLOUDY_NAME, sep=',')

# 8) Calculate weighted average flow direction for each glacier

The following code processes ice velocity rasters to determine each glacier of interest's weighted average flow direction. The rasters are subset using the terminus box shapefile using a GDAL command (**gdalwarp**) with the following syntax:

    gdalwarp -cutline path_to_terminusbox.shp -crop_to_cutline path_to_input_velocity.TIF path_to_output_velocity_at_term###.TIF

In [38]:
for BoxID in BoxIDs:
    terminus_path = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+".shp"  # set path to RGI shapefiles
    vx = 'greenland_vel_mosaic250_vx_v1.tif';vy = 'greenland_vel_mosaic250_vy_v1.tif' # names of velocity files
    
    if os.path.exists(terminus_path) == False: # if the RGI shapefile does not exist
        terminus_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"  # set the path to the box shapefile instead       
        #SET output paths
        vx_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx; vy_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy; 
    else:
        #SET output paths
        vx_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx; vy_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy; 
        
    vx_in = basepath+vx; vy_in = basepath+vy #set input paths
        
    #SET velocity subset commands and print to check it
    v_subset1 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vx_in+" "+vx_out
    v_subset2 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vy_in+" "+vy_out
#         print(v_subset_dir_cmd); print(v_subset_mag_cmd)

    #SUBSET velocity rasters
    subprocess.call(v_subset1, shell=True)
    subprocess.call(v_subset2, shell=True)
    
    print("Box"+BoxID+' done.')

Box245 done.
Box296 done.
Box220 done.
Box240 done.
Box277 done.
Box270 done.
Box246 done.


Next, these subset velocity rasters are opened using the **rasterio** package and read into arrays. They are filtered for anomalous values and the velocity magnitudes are converted into weights. Then the **numpy.average()** function is used to calculated the weighted average flow directions where the flow directions of the pixels where the highest velocities are found are weighted more. 

The resulting average flow direction will be representative of the glacier's main flow. These directions will be used to rotate the images of the glaciers so that their flow is due right.

In [39]:
#CREATE list of glacier average flow directions:
boxes = []; avg_rot = []; max_mag = []; num_cells = []

for BoxID in BoxIDs:
    rot_angles = []; max_magnitudes = []
    
    #READ velocity direction and magnitude data at terminus for each glacier into an array
    vx_name = 'greenland_vel_mosaic250_vx_v1.tif';vy_name = 'greenland_vel_mosaic250_vy_v1.tif'

    rgi_exists = 0
    for file in os.listdir(basepath+"Box"+BoxID):
        if file.startswith('RGI') and "greenland_vel" in file:
            rgi_exists = 1
    if rgi_exists == 1:      
        vx = rasterio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx_name, "r") # RGI
        vy = rasterio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy_name, "r") # RGI
    else:
        vx = rasterio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx_name, "r") # box 
        vy = rasterio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy_name, "r") # box
         
    vx_array = vx.read(); vy_array = vy.read()
    #remove no data values (-2000000000.0)
    vx_masked = vx_array[vx_array != -2000000000.0]; vy_masked = vy_array[vy_array != -2000000000.0]
        
    #CALCULATE FLOW DIRECTION
    direction = np.arctan2(vy_masked, vx_masked)*180/np.pi; 
#   print(BoxID, direction.max(), direction.min())
    #transform so any negative angles are placed on 0 to 360 scale:
    if len(direction[direction < 0]) > 0:
        direction[direction < 0] = 360.0+direction[direction < 0]
        
    #CALCULATE SPEED
    magnitude = np.sqrt((vx_masked*vx_masked) + (vy_masked*vy_masked)) 
        
    if len(direction) > 0: # if there are velocity values remaining
        # IF THERE ARE DIRECTIONS THAT HAVE A REALLY LARGE RANGE > 250.0 it's probably pointing East
        dir_range = direction.max() - direction.min()
        if dir_range > 200.0 and len(direction[direction >200]): # if large range and values above 200
            direction[direction > 180] = direction[direction > 180] - 360.0 # grab those values on a negative scale
            # CALCULATE the weighted average rotation angle
            # calculate weights (0 - 1) from magnitudes
            mag_range = magnitude.max() - magnitude.min(); stretch = 1/mag_range
            weights = stretch*(magnitude - magnitude.min())
            avg_dir = np.average(direction, weights=weights)
            max_magnitude = magnitude.max()*0.00273973 # conversion to m/d?
#           print("East!", avg_dir)
            if avg_dir < 0:
                avg_dir = avg_dir + 360.0 # transform back to 0 to 360 scale
        else:
            mag_range = magnitude.max() - magnitude.min()
            stretch = 1/mag_range
            weights = stretch*(magnitude - magnitude.min())
            avg_dir = np.average(direction, weights=weights)
    #           print(avg_dir)
            max_magnitude = magnitude.max()*0.00273973 # conversion to m/d?
    else:
        avg_dir = np.NaN ; max_magnitude = np.NaN # no velocities to calculate this with

    #APPEND to lists:
    avg_rot.append(avg_dir); max_mag.append(max_magnitude); boxes.append(BoxID); num_cells.append(len(direction))
        
velocities_df = pd.DataFrame(list(zip(boxes,avg_rot, max_mag, num_cells)), columns=['BoxID','Flow_dir', 'Max_speed', 'Pixels'])
velocities_df = velocities_df.sort_values(by='BoxID')
velocities_df = velocities_df.drop_duplicates()
velocities_df

,BoxID,Flow_dir,Max_speed,Pixels
2,220,299.409119,0.672462,838
3,240,258.306412,1.250792,39
0,245,334.956106,0.379106,224
6,246,348.945621,0.791639,244
5,270,323.513195,0.232088,18
4,277,307.169334,0.147524,11
1,296,11.153867,0.102305,194


#### Write glacier velocities to a csv file

In [42]:
VEL_NAME = 'Glacier_vel_measures_SE_focus.csv'
velocities_df.to_csv(path_or_buf = basepath+VEL_NAME, sep=',')

# 9) Rotate all images by flow direction

Read in the glacier velocity file as velocities_df if not already loaded:

In [4]:
# velocities_df = pd.read_csv(basepath+'Glacier_vel_measures_SE.csv', sep=',', dtype=str)
velocities_df = pd.read_csv(basepath+'Glacier_vel_measures_SE_focus.csv', sep=',', dtype=str, usecols=[1,2,3,4])
velocities_df = velocities_df.set_index('BoxID')
# velocities_df = velocities_df.drop(['004', '033', '235', '277', '531'])
velocities_df

,Flow_dir,Max_speed,Pixels
BoxID,,,
220,299.40911865234375,0.6724615781286621,838
240,258.30641174316406,1.250792427944336,39
245,334.9561061859131,0.3791057492289734,224
246,348.9456214904785,0.7916392478591919,244
270,323.5131950378418,0.23208783502189637,18
277,307.1693344116211,0.14752424545749665,11
296,11.1538667678833,0.10230455992904663,194


In [44]:
#make directory for rotated images in BoxID folders if it doesn't already exist
for index, row in velocities_df.iterrows():
    BoxID = index
    if os.path.exists(downloadpath+"Box"+BoxID+'/rotated_c1/'):
        print("Already exists.")
#         shutil.rmtree(downloadpath+"Box"+BoxID+'/reprojected/') # DELETE ROTATED IMAGES
        #OTHERWISE, create the folder and download into it
    else:
        os.mkdir(downloadpath+"Box"+BoxID+'/rotated_c1/')
        print("Folder made for Box"+BoxID)

Folder made for Box220
Folder made for Box240
Folder made for Box245
Folder made for Box246
Folder made for Box270
Already exists.
Folder made for Box296


In [5]:
#move rasterized terminus box into reprojected folder:
for index, row in velocities_df.iterrows():
    BoxID = index
    boxfile = 'Box'+BoxID+'_raster_cut.TIF'
    boxrasterpath = basepath+'Box'+BoxID+'/'+boxfile
    newpath = downloadpath+'Box'+BoxID+'/reprojected/'+boxfile
    shutil.copyfile(boxrasterpath, newpath)

In [6]:
#convert rasterized box into png
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *.TIF'
#     print(command)
    subprocess.call(command, shell=True)

In [8]:
#ROTATE THE IMAGES IF VELOCITIES ARE GOOD
for index, row in velocities_df.iterrows():
    BoxID = index
    print(BoxID)
    #for each file in the reprojected folder:
    for file in os.listdir(downloadpath+"Box"+BoxID+'/reprojected/'):
        if file.endswith('.png'):
            img  = Image.open(downloadpath+"Box"+BoxID+'/reprojected/'+file)
            rotated = img.rotate(-float(row['Flow_dir']))
            rotated.save(downloadpath+"Box"+BoxID+'/rotated_c1/R_'+file)

220
240
245
246
270
277
296


# 10) Resize all images to the same size (the minimum image size) 

Uses the function resize_pngs which resizes all png files within a folder to the minimum image dimensions found. Centers images when cropping around the edges.

In [9]:
for index, row in velocities_df.iterrows():
    BoxID = index
    print(BoxID)
    resizepath = downloadpath+"Box"+BoxID+'/rotated_c1/'
    resize_pngs(resizepath)

220
240
245
246
270
277
296


#### Convert these any png images to pgm for Xsmurf analysis:

In [10]:
#convert all final files to pgm
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/rotated_c1/; '+'mogrify -depth 16 -format pgm *.png'
    subprocess.call(command, shell=True)

In [22]:
# # rename the rasterized terminus box files if necessary
# for index, row in velocities_df.iterrows():
#     BoxID = index
#     files = os.listdir(downloadpath+'Box'+BoxID+'/rotated_c1/')
#     for file in files:
#         if file.startswith('R_Box'+BoxID+'_cut'):
#             rpath = downloadpath+'Box'+BoxID+'/rotated_c1/'
#             os.rename(rpath+file, rpath+'R_Box'+BoxID+'_raster_cut'+file[-4:])